In [ ]:
import batoid
import os
import yaml
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
HSC_fn = os.path.join(batoid.datadir, "hsc", "HSC3.yaml")
config = yaml.load(open(HSC_fn))
HSC_telescope = batoid.parse.parse_optic(config['opticalSystem'])

In [ ]:
LSST_fn = os.path.join(batoid.datadir, "lsst", "LSST_r2.yaml")
config = yaml.load(open(LSST_fn))
LSST_telescope = batoid.parse.parse_optic(config['opticalSystem'])

In [ ]:
DECam_fn = os.path.join(batoid.datadir, "decam", "DECam2.yaml")
config = yaml.load(open(DECam_fn))
DECam_telescope = batoid.parse.parse_optic(config['opticalSystem'])

In [ ]:
def dthdr(telescope, th, pixelSize=15, n=1.0):
    th = th * np.pi/180
    # d(pixels) / d(arcsec)
    dth = 1e-5
    rays = batoid.circularGrid(20, 5.0, 0.0, np.sin(th), 0, -np.sqrt(1-np.sin(th)**2), 32, 128, wavelength=620e-9, n=n)
    rays_x = batoid.circularGrid(20, 5.0, 0.0, np.sin(th+dth), 0, -np.sqrt(1-np.sin(th+dth)**2), 32, 128, wavelength=620e-9, n=n)
    rays_y = batoid.circularGrid(20, 5.0, 0.0, np.sin(th), np.sin(dth), -np.sqrt(1-np.sin(th)**2-np.sin(dth)**2), 32, 128, wavelength=620e-9, n=n)

    telescope.traceInPlace(rays)
    telescope.traceInPlace(rays_x)
    telescope.traceInPlace(rays_y)
    
    batoid.trimVignettedInPlace(rays)
    batoid.trimVignettedInPlace(rays_x)
    batoid.trimVignettedInPlace(rays_y)
    
    drx_dthx = (np.mean(rays_x.x) - np.mean(rays.x))/dth # m / radian
    drx_dthx *= 1e6 # micron / radian
    drx_dthx /= 206265 # micron / arcsec
    drx_dthx /= pixelSize # pixels / arcsec

    drx_dthy = (np.mean(rays_y.x) - np.mean(rays.x))/dth # m / radian
    drx_dthy *= 1e6 # micron / radian
    drx_dthy /= 206265 # micron / arcsec
    drx_dthy /= pixelSize # pixels / arcsec

    dry_dthx = (np.mean(rays_x.y) - np.mean(rays.y))/dth # m / radian
    dry_dthx *= 1e6 # micron / radian
    dry_dthx /= 206265 # micron / arcsec
    dry_dthx /= pixelSize # pixels / arcsec

    dry_dthy = (np.mean(rays_y.y) - np.mean(rays.y))/dth # m / radian
    dry_dthy *= 1e6 # micron / radian
    dry_dthy /= 206265 # micron / arcsec
    dry_dthy /= pixelSize # pixels / arcsec
    
    dr_dth = np.array([[drx_dthx, dry_dthx], [drx_dthy, dry_dthy]])
    dth_dr = np.linalg.inv(dr_dth)

    return dth_dr

In [ ]:
dthx_drx = []
dthy_dry = []
thetas = np.linspace(0.0, 0.74, 10)
for th in thetas:
    dth_dr = dthdr(HSC_telescope, th)
    dthx_drx.append(abs(dth_dr[0,0]))
    dthy_dry.append(abs(dth_dr[1,1]))
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))
ax1.plot(thetas, dthx_drx, c='b', label='radial')
ax1.plot(thetas, dthy_dry, c='r', label='tangential')
ax1.set_xlabel('radius (deg)')
ax1.set_ylabel('pixel size (arcsec)')
ax1.legend(loc='lower left')
ax1.set_title("HSC")

ax2.plot(thetas, np.array(dthx_drx)/np.array(dthy_dry))
ax2.set_xlabel('radius (deg)')
ax2.set_ylabel('b/a')
ax2.set_title("HSC")

fig.tight_layout()
fig.show()

In [ ]:
dthx_drx = []
dthy_dry = []
thetas = np.linspace(0.0, 1.75, 10)
for th in thetas:
    dth_dr = dthdr(LSST_telescope, th, pixelSize=10, n=batoid.Air())
    dthx_drx.append(abs(dth_dr[0,0]))
    dthy_dry.append(abs(dth_dr[1,1]))
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))
ax1.plot(thetas, dthx_drx, c='b', label='radial')
ax1.plot(thetas, dthy_dry, c='r', label='tangential')
ax1.set_xlabel('radius (deg)')
ax1.set_ylabel('pixel size (arcsec)')
ax1.legend(loc='lower left')
ax1.set_title("LSST")

ax2.plot(thetas, np.array(dthx_drx)/np.array(dthy_dry))
ax2.set_xlabel('radius (deg)')
ax2.set_ylabel('b/a')
ax2.set_title("LSST")

fig.tight_layout()
fig.show()

In [ ]:
dthx_drx = []
dthy_dry = []
thetas = np.linspace(0.0, 1.1, 10)
for th in thetas:
    dth_dr = dthdr(DECam_telescope, th)
    dthx_drx.append(abs(dth_dr[0,0]))
    dthy_dry.append(abs(dth_dr[1,1]))
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))
ax1.plot(thetas, dthx_drx, c='b', label='radial')
ax1.plot(thetas, dthy_dry, c='r', label='tangential')
ax1.set_xlabel('radius (deg)')
ax1.set_ylabel('pixel size (arcsec)')
ax1.legend(loc='lower left')
ax1.set_title("DECam")

ax2.plot(thetas, np.array(dthx_drx)/np.array(dthy_dry))
ax2.set_xlabel('radius (deg)')
ax2.set_ylabel('b/a')
ax2.set_title("DECam")

fig.tight_layout()
fig.show()